In [1]:
import pandas as pd

train_df = pd.read_csv("./data/rainfall_train.csv")
test_df = pd.read_csv("./data/rainfall_test.csv")

train_df = train_df.drop(columns=['Unnamed: 0', 'rainfall_train.fc_year', 'rainfall_train.fc_month',
       'rainfall_train.fc_day', 'rainfall_train.fc_hour'])
test_df = test_df.drop(columns=['Unnamed: 0', 'rainfall_test.fc_year', 'rainfall_test.fc_month',
       'rainfall_test.fc_day', 'rainfall_test.fc_hour'])

train_df = train_df[train_df["rainfall_train.class_interval"] != -999]

In [2]:
train_df= train_df[train_df["rainfall_train.class_interval"] != 0]

In [3]:
# 각 계급 별 확률로 변경
# v00 = 100 - v01, v01 = v01 - v02 .. v08 = v09 - v08
for i in range(9):
    col = "rainfall_train.v0{}"
    train_df[col.format(i)]  = train_df[col.format(i)] - train_df[col.format(i+1)] if i != 0 else 100 - train_df[col.format(i+1)]

### 단순 강수 계급

In [4]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
train_df = train_df[train_df["rainfall_train.class_interval"] != 0]
cols = ["rainfall_train.v0{}".format(i) for i in range(9)]

acc_dict = {}
# 1-9개의 클래스로 나누어야 함
# voting 하되 높은 계급 추산 - 같을 시 dh가 낮은 데이터로
# 각 클래스당 리스트(요소는 dh)를 저장하도록 설계

for y in train_df["rainfall_train.ef_year"].unique():
    train_data = train_df[train_df["rainfall_train.ef_year"] != y]
    test_data = train_df[train_df["rainfall_train.ef_year"] == y]

    model = RandomForestClassifier(n_estimators= 25)
    model.fit(train_data[cols],train_data["rainfall_train.class_interval"])
    
    test_data["pred"] = model.predict(test_data[cols])

    group = test_data.groupby(["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour","pred"])
    # pred : dh list.
    tmp = group["rainfall_train.dh"].apply(lambda x : list(x)).reset_index()
    tmp.columns = ["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour","pred","list"]
    # list : dh list
    test_data = pd.merge(test_data , tmp ,how="left",on=["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour","pred"])
    test_data["cnt"] = test_data["list"].apply(lambda x: len(x))
    # 28842

    # max cnt - flag
    tmp = test_data.groupby(by=["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour"])["cnt"].max().reset_index()
    # 4630  
    tmp.columns = ["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour", "max_cnt"]
    test_data = pd.merge(test_data, tmp, how="left", on=["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour"])
    test_data = test_data[test_data["cnt"] == test_data["max_cnt"]]
    
    tmp = test_data.groupby(by=["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour"])["list"].apply(lambda x: min([min(i)for i in x])).reset_index()
    tmp.columns = ["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour","min_dh"]
    test_data = pd.merge(test_data,tmp,how="left",on=["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour"])
    test_data = test_data[test_data["min_dh"] == test_data["rainfall_train.dh"]]
    
    acc_dict[y] = accuracy_score(test_data["rainfall_train.class_interval"],test_data["pred"])
    
    


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_61528\4221378225.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data["pred"] = model.predict(test_data[cols])
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_61528\4221378225.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data["pred"] = model.predict(test_data[cols])
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_61528\4221378225.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [5]:
acc_dict

{'A': 0.19978401727861772, 'B': 0.2083124687030546, 'C': 0.19541939321832244}

### V_median 값을 이용한 이전 값 사용

**단기적인 예측값과 장기적인 예측값**

In [6]:
# V_median 값 설정

median_list= [0,0.15,0.35,0.75,1.5,3.5,7.5,15,25,40]
train_df["v_median"] = train_df[["rainfall_train.v0{}".format(i) for i in range(10)]].apply(lambda x:sum([ median_list[idx]* (e/100) for idx,e in enumerate(x)]), axis=1)

**day_vmedian: dh 평균 값**

In [10]:
tmp = train_df.groupby(by=["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour"])["v_median"].mean().reset_index()
tmp.columns = ["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour","day_vmedian"]
train_df = pd.merge(train_df, tmp, how="left",on=["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour"])

In [8]:
acc_dict = {}
cols = ["rainfall_train.v0{}".format(i) for i in range(9)] + ["day_vmedian"]

# 1-9개의 클래스로 나누어야 함
# voting 하되 높은 계급 추산 - 같을 시 dh가 낮은 데이터로
# 각 클래스당 리스트(요소는 dh)를 저장하도록 설계

for y in train_df["rainfall_train.ef_year"].unique():
    train_data = train_df[train_df["rainfall_train.ef_year"] != y]
    test_data = train_df[train_df["rainfall_train.ef_year"] == y]

    tmp = train_data.groupby(by=["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour"])["v_median"].mean().reset_index()
    tmp.columns = ["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour","day_vmedian"]
    train_data = pd.merge(train_data, tmp, how="left",on=["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour"])

    tmp = test_data.groupby(by=["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour"])["v_median"].mean().reset_index()
    tmp.columns = ["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour","day_vmedian"]
    test_data = pd.merge(test_data, tmp, how="left",on=["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour"])


    model = RandomForestClassifier(n_estimators= 25)
    model.fit(train_data[cols],train_data["rainfall_train.class_interval"])
    
    test_data["pred"] = model.predict(test_data[cols])

    group = test_data.groupby(["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour","pred"])
    # pred : dh list.
    tmp = group["rainfall_train.dh"].apply(lambda x : list(x)).reset_index()
    tmp.columns = ["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour","pred","list"]
    # list : dh list
    test_data = pd.merge(test_data , tmp ,how="left",on=["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour","pred"])
    test_data["cnt"] = test_data["list"].apply(lambda x: len(x))
    # 28842

    # max cnt - flag
    tmp = test_data.groupby(by=["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour"])["cnt"].max().reset_index()
    # 4630  
    tmp.columns = ["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour", "max_cnt"]
    test_data = pd.merge(test_data, tmp, how="left", on=["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour"])
    test_data = test_data[test_data["cnt"] == test_data["max_cnt"]]
    
    tmp = test_data.groupby(by=["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour"])["list"].apply(lambda x: min([min(i)for i in x])).reset_index()
    tmp.columns = ["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour","min_dh"]
    test_data = pd.merge(test_data,tmp,how="left",on=["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour"])
    test_data = test_data[test_data["min_dh"] == test_data["rainfall_train.dh"]]
    
    acc_dict[y] = accuracy_score(test_data["rainfall_train.class_interval"],test_data["pred"])

In [9]:
acc_dict

{'A': 0.21468682505399567, 'B': 0.20405608412618928, 'C': 0.2099940511600238}

**단기적인 예측 값(7일)**


In [20]:
tmp = pd.DataFrame()
for y in  train_df["rainfall_train.ef_year"].unique():
    for stn in train_df["rainfall_train.stn4contest"].unique():
        df = train_df[(train_df["rainfall_train.stn4contest"] == stn) & (train_df["rainfall_train.ef_year"] == y)]
        df["prev_vmedian"] = df.rolling(window=7)["day_vmedian"].mean()
        tmp = pd.concat([tmp,df],axis=0)
train_df = tmp

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_61528\1627897360.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["prev_vmedian"] = df.rolling(window=7)["day_vmedian"].mean()
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_61528\1627897360.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["prev_vmedian"] = df.rolling(window=7)["day_vmedian"].mean()
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_61528\1627897360.py:5: SettingWithCopyWarning: 
A value is trying to be set on

In [23]:
acc_dict = {}
cols = ["rainfall_train.v0{}".format(i) for i in range(9)] + ["day_vmedian","prev_vmedian"]

# 1-9개의 클래스로 나누어야 함
# voting 하되 높은 계급 추산 - 같을 시 dh가 낮은 데이터로
# 각 클래스당 리스트(요소는 dh)를 저장하도록 설계

for y in train_df["rainfall_train.ef_year"].unique():
    train_data = train_df[train_df["rainfall_train.ef_year"] != y]
    test_data = train_df[train_df["rainfall_train.ef_year"] == y]

    model = RandomForestClassifier(n_estimators= 25)
    model.fit(train_data[cols],train_data["rainfall_train.class_interval"])
    
    test_data["pred"] = model.predict(test_data[cols])

    group = test_data.groupby(["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour","pred"])
    # pred : dh list.
    tmp = group["rainfall_train.dh"].apply(lambda x : list(x)).reset_index()
    tmp.columns = ["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour","pred","list"]
    # list : dh list
    test_data = pd.merge(test_data , tmp ,how="left",on=["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour","pred"])
    test_data["cnt"] = test_data["list"].apply(lambda x: len(x))
    # 28842

    # max cnt - flag
    tmp = test_data.groupby(by=["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour"])["cnt"].max().reset_index()
    # 4630  
    tmp.columns = ["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour", "max_cnt"]
    test_data = pd.merge(test_data, tmp, how="left", on=["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour"])
    test_data = test_data[test_data["cnt"] == test_data["max_cnt"]]
    
    tmp = test_data.groupby(by=["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour"])["list"].apply(lambda x: min([min(i)for i in x])).reset_index()
    tmp.columns = ["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour","min_dh"]
    test_data = pd.merge(test_data,tmp,how="left",on=["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour"])
    test_data = test_data[test_data["min_dh"] == test_data["rainfall_train.dh"]]
    
    acc_dict[y] = accuracy_score(test_data["rainfall_train.class_interval"],test_data["pred"])

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_61528\2539794428.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data["pred"] = model.predict(test_data[cols])
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_61528\2539794428.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data["pred"] = model.predict(test_data[cols])
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_61528\2539794428.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [24]:
acc_dict

{'A': 0.21144708423326133, 'B': 0.19879819729594392, 'C': 0.21207614515169543}

**장기적인 예측 값(30일)**

In [38]:
tmp = pd.DataFrame()
for y in  train_df["rainfall_train.ef_year"].unique():
    for stn in train_df["rainfall_train.stn4contest"].unique():
        df = train_df[(train_df["rainfall_train.stn4contest"] == stn) & (train_df["rainfall_train.ef_year"] == y)]
        df["post_vmedian"] = df.rolling(window=30)["day_vmedian"].mean()
        tmp = pd.concat([tmp,df],axis=0)
train_df = tmp

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_61528\372841459.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["post_vmedian"] = df.rolling(window=30)["day_vmedian"].mean()


In [39]:
acc_dict = {}
cols = ["rainfall_train.v0{}".format(i) for i in range(9)] + ["day_vmedian","post_vmedian"]

# 1-9개의 클래스로 나누어야 함
# voting 하되 높은 계급 추산 - 같을 시 dh가 낮은 데이터로
# 각 클래스당 리스트(요소는 dh)를 저장하도록 설계

for y in train_df["rainfall_train.ef_year"].unique():
    train_data = train_df[train_df["rainfall_train.ef_year"] != y]
    test_data = train_df[train_df["rainfall_train.ef_year"] == y]

    model = RandomForestClassifier(n_estimators= 25)
    model.fit(train_data[cols],train_data["rainfall_train.class_interval"])
    
    test_data["pred"] = model.predict(test_data[cols])

    group = test_data.groupby(["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour","pred"])
    # pred : dh list.
    tmp = group["rainfall_train.dh"].apply(lambda x : list(x)).reset_index()
    tmp.columns = ["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour","pred","list"]
    # list : dh list
    test_data = pd.merge(test_data , tmp ,how="left",on=["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour","pred"])
    test_data["cnt"] = test_data["list"].apply(lambda x: len(x))
    # 28842

    # max cnt - flag
    tmp = test_data.groupby(by=["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour"])["cnt"].max().reset_index()
    # 4630  
    tmp.columns = ["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour", "max_cnt"]
    test_data = pd.merge(test_data, tmp, how="left", on=["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour"])
    test_data = test_data[test_data["cnt"] == test_data["max_cnt"]]
    
    tmp = test_data.groupby(by=["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour"])["list"].apply(lambda x: min([min(i)for i in x])).reset_index()
    tmp.columns = ["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour","min_dh"]
    test_data = pd.merge(test_data,tmp,how="left",on=["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour"])
    test_data = test_data[test_data["min_dh"] == test_data["rainfall_train.dh"]]
    
    acc_dict[y] = accuracy_score(test_data["rainfall_train.class_interval"],test_data["pred"])

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_61528\1493450800.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data["pred"] = model.predict(test_data[cols])
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_61528\1493450800.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data["pred"] = model.predict(test_data[cols])
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_61528\1493450800.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [40]:
acc_dict

{'A': 0.21403887688984882, 'B': 0.2083124687030546, 'C': 0.21267102914931588}

**장/단기 예측**

In [41]:
acc_dict = {}
cols = ["rainfall_train.v0{}".format(i) for i in range(9)] + ["day_vmedian","prev_vmedian","post_vmedian"]

# 1-9개의 클래스로 나누어야 함
# voting 하되 높은 계급 추산 - 같을 시 dh가 낮은 데이터로
# 각 클래스당 리스트(요소는 dh)를 저장하도록 설계

for y in train_df["rainfall_train.ef_year"].unique():
    train_data = train_df[train_df["rainfall_train.ef_year"] != y]
    test_data = train_df[train_df["rainfall_train.ef_year"] == y]

    model = RandomForestClassifier(n_estimators= 25)
    model.fit(train_data[cols],train_data["rainfall_train.class_interval"])
    
    test_data["pred"] = model.predict(test_data[cols])

    group = test_data.groupby(["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour","pred"])
    # pred : dh list.
    tmp = group["rainfall_train.dh"].apply(lambda x : list(x)).reset_index()
    tmp.columns = ["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour","pred","list"]
    # list : dh list
    test_data = pd.merge(test_data , tmp ,how="left",on=["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour","pred"])
    test_data["cnt"] = test_data["list"].apply(lambda x: len(x))
    # 28842

    # max cnt - flag
    tmp = test_data.groupby(by=["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour"])["cnt"].max().reset_index()
    # 4630  
    tmp.columns = ["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour", "max_cnt"]
    test_data = pd.merge(test_data, tmp, how="left", on=["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour"])
    test_data = test_data[test_data["cnt"] == test_data["max_cnt"]]
    
    tmp = test_data.groupby(by=["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour"])["list"].apply(lambda x: min([min(i)for i in x])).reset_index()
    tmp.columns = ["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour","min_dh"]
    test_data = pd.merge(test_data,tmp,how="left",on=["rainfall_train.stn4contest", "rainfall_train.ef_year","rainfall_train.ef_month","rainfall_train.ef_day","rainfall_train.ef_hour"])
    test_data = test_data[test_data["min_dh"] == test_data["rainfall_train.dh"]]
    
    acc_dict[y] = accuracy_score(test_data["rainfall_train.class_interval"],test_data["pred"])

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_61528\1624593439.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data["pred"] = model.predict(test_data[cols])
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_61528\1624593439.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data["pred"] = model.predict(test_data[cols])
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_61528\1624593439.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [42]:
acc_dict

{'A': 0.2125269978401728, 'B': 0.2015523284927391, 'C': 0.2070196311719215}

In [43]:
cols = ["rainfall_train.class_interval", "day_vmedian","prev_vmedian","post_vmedian"]

train_df[cols].corr()

,rainfall_train.class_interval,day_vmedian,prev_vmedian,post_vmedian
rainfall_train.class_interval,1.000000,0.457771,0.214989,0.206721
day_vmedian,0.457771,1.000000,0.636249,0.571135
prev_vmedian,0.214989,0.636249,1.000000,0.797013
post_vmedian,0.206721,0.571135,0.797013,1.000000


In [ ]:
# 전체
acc_dict

{'A': 0.19654427645788336, 'B': 0.2013019529293941, 'C': 0.1998810232004759}

In [35]:
model.feature_importances_

array([0.10067685, 0.07654692, 0.08681202, 0.0875673 , 0.08734619,
       0.08565881, 0.07117142, 0.05541418, 0.02812769, 0.16066507,
       0.16001355])